In [1]:
import json
import os
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from multiprocessing import Pool

In [2]:
DATASET_FOLDER = "./dataset"
train_data = os.path.join(DATASET_FOLDER, 'train_data.parquet')
item_data = os.path.join(DATASET_FOLDER, 'items_static_metadata_full.jl')
test_data = os.path.join(DATASET_FOLDER, 'test_data.csv')

In [43]:
df_train = pd.read_parquet(train_data, engine='fastparquet')
df_train['datetime'] = pd.to_datetime(df_train['date'])

In [3]:
df_item = pd.read_json(item_data, lines=True)

In [4]:
df_item

,item_domain_id,item_id,item_title,site_id,sku,product_id,product_family_id
0,MLB-SNEAKERS,492155,Tênis Masculino Olympikus Cyber Barato Promoçao,MLB,0,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB,1,None,None
2,MLM-NECKLACES,69847,5 Collares Plateados Dama Gargantilla Choker -...,MLM,2,None,None
3,MLM-RINGS,298603,Lindo Anillo De Bella Crepusculo Twilight Prom...,MLM,3,None,None
4,MLB-WEBCAMS,345949,Webcam Com Microfone Hd 720p Knup Youtube Pc V...,MLB,4,None,None
...,...,...,...,...,...,...,...
660911,MLB-CELLPHONE_PARTS,320792,2 Parafuso Pentalobe Torx iPhone 5 5c 5s 6 6s ...,MLB,660911,None,None
660912,MLM-AUTOMOTIVE_EMBLEMS,90441,Emblema Circular Wolfsburg Edition 5cm,MLM,660912,None,None
660913,MLB-SOFA_AND_FUTON_COVERS,202580,Kit Capa De Sofá Coladinha Jogo 2e3 Lugares Ma...,MLB,660913,None,None
660914,MLB-SNEAKERS,490874,Calçado Sapatênis Clássico Baltic 234 Ollie,MLB,660914,None,None


In [5]:
df_train

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,datatime
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2021-02-01
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2021-02-02
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2021-02-03
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2021-02-04
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2021-02-05
...,...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.00,ARG,classic,drop_off,free_shipping,267.710767,2021-03-31
37660275,6707,2021-03-31,0,26999.00,ARG,classic,cross_docking,free_shipping,266.083333,2021-03-31
37660276,170355,2021-03-31,0,3400.00,ARG,classic,drop_off,paid_shipping,0.252633,2021-03-31
37660277,246568,2021-03-31,0,6289.00,ARG,classic,fulfillment,free_shipping,135.416667,2021-03-31


In [47]:
df_first_entry = df_train[df_train['sku'].diff() != 0]
df_first_entry

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,datetime
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2021-02-01
59,645793,2021-02-01,0,134.23,REA,classic,fulfillment,free_shipping,1440.000000,2021-02-01
118,77402,2021-02-01,0,59.50,REA,classic,cross_docking,paid_shipping,1440.000000,2021-02-01
177,58546,2021-02-01,0,157.00,REA,classic,cross_docking,free_shipping,1440.000000,2021-02-01
236,99516,2021-02-01,0,22.90,REA,premium,fulfillment,paid_shipping,1440.000000,2021-02-01
...,...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.00,ARG,classic,drop_off,free_shipping,267.710767,2021-03-31
37660275,6707,2021-03-31,0,26999.00,ARG,classic,cross_docking,free_shipping,266.083333,2021-03-31
37660276,170355,2021-03-31,0,3400.00,ARG,classic,drop_off,paid_shipping,0.252633,2021-03-31
37660277,246568,2021-03-31,0,6289.00,ARG,classic,fulfillment,free_shipping,135.416667,2021-03-31


In [35]:
len(df_first_entry['sku'].unique()) == len(df_first_entry)

True

In [54]:
len(df_first_entry['sku'].unique()) == len(df_item)

True

In [52]:
df_first_entry['datetime'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

2021-02-01    91.42%
2021-02-04     0.37%
2021-02-02     0.33%
2021-02-03     0.29%
2021-02-05     0.26%
2021-02-11     0.26%
2021-02-10     0.25%
2021-02-12     0.24%
2021-02-08     0.23%
2021-02-09     0.23%
2021-02-24     0.23%
2021-02-17     0.22%
2021-03-05     0.22%
2021-02-25     0.22%
2021-02-18     0.21%
2021-02-23      0.2%
2021-02-22      0.2%
2021-03-03      0.2%
2021-03-04     0.19%
2021-02-16     0.19%
2021-02-19     0.18%
2021-03-02     0.18%
2021-02-26     0.18%
2021-02-15     0.18%
2021-03-01     0.18%
2021-03-11     0.16%
2021-03-10     0.16%
2021-03-08     0.15%
2021-02-06     0.14%
2021-03-09     0.14%
2021-03-12     0.13%
2021-03-18     0.13%
2021-03-22     0.12%
2021-03-16     0.12%
2021-03-17     0.12%
2021-03-23     0.11%
2021-03-15     0.11%
2021-03-19      0.1%
2021-03-24      0.1%
2021-02-13      0.1%
2021-03-25     0.09%
2021-03-26     0.09%
2021-03-30     0.08%
2021-03-31     0.08%
2021-03-29     0.08%
2021-02-07     0.08%
2021-02-20     0.08%
2021-02-27   

In [67]:
sku_split = []
for i in tqdm(range(len(df_first_entry)-1)):
    start_index = df_first_entry.index[i]
    end_index = df_first_entry.index[i+1]
    sku_split.append(df_train.iloc[start_index:end_index])
sku_split.append(df_train.iloc[end_index:])

  0%|          | 0/660915 [00:00<?, ?it/s]

In [132]:
def create_analysis_features(df):
    sku = df['sku'].iloc[0]
    new_row = df_item.loc[sku].to_dict()

    new_row['count'] = len(df)
    new_row['date_first'] = df['datetime'].iloc[0]
    new_row['date_last'] = df['datetime'].iloc[-1]
    new_row['date_diff'] = (new_row['date_last'] - new_row['date_first']).days

    for quantity in ['sold_quantity', 'current_price', 'minutes_active']:
        new_row[quantity+'_first'] = df[quantity].iloc[0]
        new_row[quantity+'_last'] = df[quantity].iloc[-1]
        new_row[quantity+'_sum'] = df[quantity].sum()
        new_row[quantity+'_mean'] = df[quantity].mean()
        new_row[quantity+'_std'] = df[quantity].std()
        new_row[quantity+'_min'] = df[quantity].min()
        new_row[quantity+'_max'] = df[quantity].max()
        new_row[quantity+'_mode'] = df[quantity].mode().iloc[0]
        new_row[quantity+'_mode_tx'] = df[quantity].value_counts().iloc[0]/new_row['count']

    for quantity in ['currency', 'listing_type', 'shipping_logistic_type', 'shipping_payment']:
        new_row[quantity+'_first'] = df[quantity].iloc[0]
        new_row[quantity+'_last'] = df[quantity].iloc[-1]
        new_row[quantity+'_mode'] = df[quantity].mode().iloc[0]
        new_row[quantity+'_mode_tx'] = df[quantity].value_counts().iloc[0]/new_row['count']

    new_row['sold_quantity_series'] = df['sold_quantity'].to_json(orient='values')
    return new_row

In [144]:
sku_data = []
with Pool(100) as p:
    for data in tqdm(p.imap(create_analysis_features, sku_split), total=len(sku_split)):
        sku_data.append(data)

  0%|          | 0/660916 [00:00<?, ?it/s]

In [149]:
df_sku = pd.DataFrame(sku_data)

In [187]:
for quantity in ['currency', 'listing_type', 'shipping_logistic_type', 'shipping_payment']:
    df_sku[quantity+'_first'] = df_sku[quantity+'_first'].astype("category")
    df_sku[quantity+'_last'] = df_sku[quantity+'_last'].astype("category")
    df_sku[quantity+'_mode'] = df_sku[quantity+'_mode'].astype("category")

In [147]:
if not os.path.exists('dataset/processed'):
    os.mkdir('dataset/processed')

In [ ]:
from fastparquet import write
write('dataset/processed/sku_feature_data.parquet', df_sku)

In [199]:
df_sku.to_csv('dataset/processed/sku_feature_data.csv', index=False)

In [201]:
df_sku.describe(include='all').transpose()

/opt/conda/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.
/opt/conda/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
item_domain_id,660913,8408,MLB-SUPPLEMENTS,13638,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
item_id,660916,NaN,NaN,NaN,NaT,NaT,259141,149446,0,129868,259206,388517,517895
item_title,660916,478175,Case Acrigel iPhone Galaxy Huawei Xiaomi Moto ...,167,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
site_id,660916,3,MLB,336334,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sku,660916,NaN,NaN,NaN,NaT,NaT,330458,190790,0,165229,330458,495686,660915
product_id,29668,15863,MLB6125887,36,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_family_id,83184,29600,MLB12287867,262,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
count,660916,NaN,NaN,NaN,NaT,NaT,56.9819,8.09463,1,59,59,59,59
date_first,660916,59,2021-02-01 00:00:00,604230,2021-02-01,2021-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date_last,660916,1,2021-03-31 00:00:00,660916,2021-03-31,2021-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
